# HTTPS & TLS
von Jens Gräfe, Matthias Rinker & Jan Kühnemund

# Inhaltsverzeichnis

# HTTPS

# HTTPS Demo

In [ ]:
import ssl
import socket
import OpenSSL

# Set the target website
website = "www.google.com"

# Check the certificate of the website
context = ssl.create_default_context()
s = socket.create_connection((website, 443))
ssl_sock = context.wrap_socket(s, server_hostname=website)
cert = ssl_sock.getpeercert()

# Print the certificate
print(cert)

# HTTPS ist HTTP über TLS

HTTP nutzt als Transport Protokoll TCP. HTTPS setzt stattdessen auf TLS auf, welches auch TCP nutzt. Dadurch ist das komplette HTTP Paket verschlüsselt. (rfc2818[2][2.2])

# HTTP Paketaufbau

In [4]:
from diagrams import Diagram
from diagrams.onprem.client import Client
from diagrams.onprem.network import Tyk

with Diagram("HTTP Header Format"):
    client = Client("Client")
    http = Tyk("HTTP")
    client >> http

# HTTP Ablauf

| method | scheme | host | path | accept | user-agent |
|--------|--------|------|------|--------|------------|
| GET    | https  | www.example.com | / | text/html | curl/7.54.0 |

# TLS
Transfler Layer Security (TLS) ist ein Protokoll zur Verschlüsselung von Daten. Es ist ein Nachfolger von SSL (Secure Sockets Layer). TLS ist ein Protokoll, welches auf TCP aufbaut. (rfc2818[2][2.1])

Es ist in Schicht 5 des OSI Schichtenmodells angesiedelt und teilt sich auf 2 Unterschichten auf.

| Handshake | Alert | Change Cipher | Application Data |
|-----------|-------|---------------|------------------|
|                 TLS Record Protocol                  |



## Record Protocol

Das record prtocol ist die unterste Schicht von TLS. Es ist für die Verschlüsselung und die Authentifizierung der Daten zuständig. (rfc8446[5]). Es teilt die Daten in verarbeitbare Blöcke auf, schützt diese und überträgt das Ergebnis. Auf der Gegenseite werden die empfangenen Daten überprüft, entschlüsselt, zusammengesetzt und an höhere OSI Layer weitergegeben.

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-c3ow">Byte</th>
    <th class="tg-c3ow">+0</th>
    <th class="tg-c3ow">+1</th>
    <th class="tg-c3ow">+2</th>
    <th class="tg-baqh">+3</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-c3ow">0</td>
    <td class="tg-c3ow">Content type<br></td>
    <td class="tg-0pky" colspan="3"></td>
  </tr>
  <tr>
    <td class="tg-baqh">1..4</td>
    <td class="tg-baqh" colspan="2">Version</td>
    <td class="tg-baqh" colspan="2">Length</td>
  </tr>
  <tr>
    <td class="tg-baqh">5..n</td>
    <td class="tg-baqh" colspan="4">Payload</td>
  </tr>
  <tr>
    <td class="tg-baqh">n..m</td>
    <td class="tg-baqh" colspan="4">MAC</td>
  </tr>
  <tr>
    <td class="tg-baqh">m..p</td>
    <td class="tg-baqh" colspan="4">Padding (Block ciphers only)</td>
  </tr>
</tbody>
</table>

## Handshake

# Change Cipher Spec
Das Change Cipher Spec Byte wird nur aus Kompatibilitätsgründen weiter verwendet. IM TLS 1.3 Standard muss ein erkannt und verworfen werden damit keine Fehlermeldungen entstehen.

## Alert Protocol

Das Alert Protocol überträgt Fehlermeldung und Übertragunsgbeendigungsinformationen.

## Application Data Protocol

Application Data Messages werden immer verschlüsselt und sind für TLS undurchsichtig. Sie beinhalten TLSPlaintext Structs welche mittels record protection Funktionen in TLSCiphertext Strukturen umgewandelt werden.

### Authenticated Encryption with Associated Data (AEAD) RFC5116

AEAD ist ein Verfahren zur Authentifizierung und Verschlüsselung von Daten. Es nutzt dabei eine Authentifizierungs- und Verschlüsselungsfunktion. Diese Funktion ist ein Verfahren welches sowohl Authentifizierung als auch Verschlüsselung in einem Schritt durchführt.

Vorher wurden Verschlüsselung und Authentifizierung häufig getrennt durchgeführt. Ein oft genutzter Authentifizierung Algorithmus war die Nutzung von MACs.

#### Message Authentication Code

Ein Message Authentication Code (MAC) ist ein Verfahren zur Authentifizierung von Daten. Es wird verwendet um die Integrität von Daten zu überprüfen. Es wird ein Hashwert aus den Daten berechnet und mit einem Schlüssel verschlüsselt. Der verschlüsselte Hashwert wird mit den Daten übertragen. Der Empfänger berechnet den Hashwert aus den Daten und entschlüsselt den Hashwert. Wenn die beiden Hashwerte übereinstimmen ist die Integrität der Daten gewährleistet. (rfc2104[3])


## Cipher Suite

In der TLS Version 1.3 werden als unsicher erachtete Cipher Suites der vorherigen Versionen entfernt. Ein TLS 1.3 Client ist daher mit einem TLS 1.2 Client nicht mehr zwingend kompatibel.

TLS 1.3 Cipher Suites sind anders aufgebaut als TLS 1.2 Cipher Suites, daher können diese nicht untereinander verwendet werden.

### RFC8446 9.1

#### Verpflichtend

- TLS_AES_128_GCM_SHA256

#### Optional

- TLS_AES_256_GCM_SHA384
- TLS_CHACHA20_POLY1305_SHA256 (RFC8439)

### OpenSSL https://ciphersuite.info/cs/?software=openssl&tls=tls13

- TLS_AES_128_GCM_SHA256 (Empfohlen)
- TLS_AES_256_GCM_SHA384 (Empfohlen)
- TLS_AES_128_CCM_SHA256 (Hinreichend Sicher)

# Geschichte von TLS

# TLS 1.3